# A/B Testing
## Create the endpoint

In [ ]:
import boto3, time
sagemaker_client = boto3.client('sagemaker')

underfit_model_name = f"xgboost-model-underfit"
normal_model_name = f"xgboost-model-normal"
overfit_model_name = f"xgboost-model-overfit"

# Define the three models
variant_underfit = {
  'VariantName': 'ModelUnderfit',
  'ModelName': underfit_model_name,
  'InitialVariantWeight': 0.33,
  'InitialInstanceCount': 1,
  'InstanceType': 'ml.m5.large'
}
variant_normal = {
  'VariantName': 'ModelNormal',
  'ModelName': normal_model_name,
  'InitialVariantWeight': 0.34,
  'InitialInstanceCount': 1,
  'InstanceType': 'ml.m5.large'
}
variant_overfit = {
  'VariantName': 'ModelOverfit',
  'ModelName': overfit_model_name,
  'InitialVariantWeight': 0.33,
  'InitialInstanceCount': 1,
  'InstanceType': 'ml.m5.large'
}

# Create endpoint configuration
endpoint_config_name = 'abc-test-endpoint-config'
sagemaker_client.create_endpoint_config(
  EndpointConfigName=endpoint_config_name,
  ProductionVariants=[variant_underfit, variant_normal, variant_overfit]
)

In [ ]:
# Deploy the endpoint
endpoint_name = 'abc-test-endpoint'
sagemaker_client.create_endpoint(
  EndpointName=endpoint_name,
  EndpointConfigName=endpoint_config_name
)

# Wait for endpoint to be ready
print("Waiting for endpoint to be ready...")
while True:
    response = sagemaker_client.describe_endpoint(EndpointName=endpoint_name)
    status = response["EndpointStatus"]
    if status in ["InService", "Failed"]:
        print(f"Endpoint Status: {status}")
        break
    time.sleep(10)

# Check if deployment was successful
if status != "InService":
    raise Exception(f"Deployment failed with status: {status}")

print(f"Model deployed successfully at endpoint: {endpoint_config_name}")

# Random Predictions
Generate random predictions sent to the endpoint, SageMaker will handle sharing the queries across the models using the pre-defined variant weight in the configuration above.

[https://docs.aws.amazon.com/sagemaker/latest/APIReference/API_runtime_InvokeEndpoint.html](https://docs.aws.amazon.com/sagemaker/latest/APIReference/API_runtime_InvokeEndpoint.html)
```
POST /endpoints/EndpointName/invocations HTTP/1.1
Content-Type: ContentType
Accept: Accept
X-Amzn-SageMaker-Custom-Attributes: CustomAttributes
X-Amzn-SageMaker-Target-Model: TargetModel
X-Amzn-SageMaker-Target-Variant: TargetVariant
X-Amzn-SageMaker-Target-Container-Hostname: TargetContainerHostname
X-Amzn-SageMaker-Inference-Id: InferenceId
X-Amzn-SageMaker-Enable-Explanations: EnableExplanations
X-Amzn-SageMaker-Inference-Component: InferenceComponentName
X-Amzn-SageMaker-Session-Id: SessionId
```

In [ ]:
import random
sagemaker_runtime = boto3.client('sagemaker-runtime')

total_predictions = 25

def random_data():
    age = random.randint(18, 80) 
    edu = random.randint(0, 4)
    income = random.randint(20000, 99999) 
    return f"{age},{edu},{income}"

for _ in range(total_predictions):
    data = random_data()
    for i in range(3):
        response = sagemaker_runtime.invoke_endpoint(
            EndpointName=endpoint_name,
            ContentType='text/csv',
            Body=data
        )
        
        responding_variant = response['ResponseMetadata']['HTTPHeaders'].get('x-amzn-invoked-production-variant', 'Unknown')
        print(f"{_}[{i}]:  Prediction for {data}\tresult: {response['Body'].read().decode('utf-8').strip()}\tvariant: {responding_variant}")